# library

In [80]:
from sqlalchemy import create_engine
from sqlalchemy.exc import OperationalError
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy.orm import session

import pandas as pd
from sqlalchemy.sql import text
import pandas as pd

# global variables

In [81]:
dbname = 'test'
user = 'user_test'
password = 'Password123'
host = 'redshift-cluster-datawarehouse.cew9a5azwld4.us-east-1.redshift.amazonaws.com'
port = 5439
protocol = 'postgresql'

In [82]:
Base = declarative_base()

class TrebleDBTest:
    def __init__(self, protocol, user, password, url, port, database):
        self.protocol = protocol
        self.user = user
        self.password = password
        self.url = url
        self.port = port
        self.database = database
        self.engine = None
        self.session = None

    def _build_connection_string(self):
        return f"{self.protocol}://{self.user}:{self.password}@{self.url}:{self.port}/{self.database}"

    def connect(self):
        connection_string = self._build_connection_string()
        self.engine = create_engine(connection_string)
        self.session = scoped_session(sessionmaker(bind=self.engine))
        print("Connection successful to Treble.AI datawarehouse!")

    def close(self):
        if self.session:
            self.session.remove()
        if self.engine:
            self.engine.dispose()

In [83]:
conn_params = {
    "protocol": protocol,
    "user": user,
    "password": password,
    "url": host,
    "port": port,
    "database": dbname
}

db_connection = TrebleDBTest(**conn_params)
db_connection.connect()

db_connection.close()



Connection successful to Treble.AI datawarehouse!


In [ ]:
def inspect_database(engine):
    # Use the inspector to get table details
    inspector = reflection.Inspector.from_engine(engine)

    # Fetch table names
    tables = inspector.get_table_names()

    print("Tables in the database:")
    for table in tables:
        print(table)

        # Fetch columns in the table
        columns = inspector.get_columns(table)
        print("Columns:")
        for column in columns:
            print(f"  - {column['name']} ({column['type']})")

        # Fetch foreign key relations
        fks = inspector.get_foreign_keys(table)
        if fks:
            print("Foreign Key Relations:")
            for fk in fks:
                print(f"  - Constraint Name: {fk['name']}")
                print(f"    Source: {fk['constrained_columns']}")
                print(f"    References: {fk['referred_table']}({', '.join(fk['referred_columns'])})")
        print("\n")

# Usage
engine = create_connection()
inspect_database(engine)

# Close the connection
engine.dispose()